In [1]:
# test calibration using scipy.optimize

In [2]:
import numpy as np
import scipy.optimize as opt

from openptv_python.calibration import Calibration
from openptv_python.imgcoord import image_coordinates
from openptv_python.orientation import (
    external_calibration,
    full_calibration
)
from openptv_python.parameters import ControlPar, OrientPar, read_control_par
from openptv_python.tracking_frame_buf import Target
from openptv_python.trafo import arr_metric_to_pixel
from openptv_python.imgcoord import img_coord
from openptv_python.trafo import pixel_to_metric
from openptv_python.tracking_frame_buf import Target


In [3]:

control_file_name = "testing_folder/corresp/control.par"
# self.control = ControlPar(4)
control = read_control_par(control_file_name)

orient_par_file_name = "testing_folder/corresp/orient.par"
orient_par = OrientPar().from_file(orient_par_file_name)

cal = Calibration().from_file(
    "testing_folder/calibration/cam1.tif.ori",
    "testing_folder/calibration/cam1.tif.addpar",
)
orig_cal = Calibration().from_file(
    "testing_folder/calibration/cam1.tif.ori",
    "testing_folder/calibration/cam1.tif.addpar",
)


# def test_external_calibration(self):
"""External calibration using clicked points."""
ref_pts = np.array(
    [
        [-40.0, -25.0, 8.0],
        [40.0, -15.0, 0.0],
        [40.0, 15.0, 0.0],
        [40.0, 0.0, 8.0],
    ]
)

# Fake the image points by back-projection
targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, control.mm),
    control,
)

# Jigg the fake detections to give raw_orient some challenge.
targets[:, 1] -= 0.1

external_calibration(cal, ref_pts, targets, control)

np.testing.assert_array_almost_equal(
    cal.get_angles(), orig_cal.get_angles(), decimal=3
)
np.testing.assert_array_almost_equal(
    cal.get_pos(), orig_cal.get_pos(), decimal=3
)


_, _, _ = full_calibration(
            cal,
            ref_pts,
            targets,
            control,
            orient_par
            )

np.testing.assert_array_almost_equal(
    cal.get_angles(), orig_cal.get_angles(), decimal=3
)
np.testing.assert_array_almost_equal(
    cal.get_pos(), orig_cal.get_pos(), decimal=3
)

In [4]:
control_file_name = "testing_folder/corresp/control.par"
control = read_control_par(control_file_name)

orient_par_file_name = "testing_folder/corresp/orient.par"
orient_par = OrientPar().from_file(orient_par_file_name)

cal = Calibration().from_file(
    "testing_folder/calibration/cam1.tif.ori",
    "testing_folder/calibration/cam1.tif.addpar",
)
orig_cal = Calibration().from_file(
    "testing_folder/calibration/cam1.tif.ori",
    "testing_folder/calibration/cam1.tif.addpar")

In [5]:
ref_pts = np.array(
    [
        [-40.0, -25.0, 8.0],
        [40.0, -15.0, 0.0],
        [40.0, 15.0, 0.0],
        [40.0, 0.0, 8.0],
    ]
)

# Fake the image points by back-projection
targets = arr_metric_to_pixel(
    image_coordinates(ref_pts, cal, control.mm),
    control,
)

cal.set_pos(np.array([0, 0, 100]))
cal.set_angles(np.array([0, 0, 0]))

# Jigg the fake detections to give raw_orient some challenge.
targets[:, 1] -= 0.1

In [6]:
targs = [Target() for _ in targets]

for ptx, pt in enumerate(targets):
    targs[ptx].x = pt[0]
    targs[ptx].y = pt[1]

In [7]:
# def residual(calibration_array, ref_pts, targs, control, cc):
#     # print(calibration_array)
#     # print(ref_pts)
#     # print(targs)
#     # print(control)
#     # print(calibration_array)
    
#     c = Calibration()
#     c.set_pos(calibration_array[:3])
#     c.set_angles(calibration_array[3:])
#     c.int_par.cc = cc
#     c.update_rotation_matrix()
    
    
#     # print(f"{c.get_pos()=}")
                
#     residual = 0
#     for i in range(len(targs)):
#         xc, yc = pixel_to_metric(targs[i].x, targs[i].y, control)
#         # print(f"{xc=}, {yc=} mm")
        
#         xp, yp = img_coord(ref_pts[i], c, control.mm)
#         # print(f"{xp=}, {yp=} mm")
#         residual += ((xc - xp)**2 + (yc - yp)**2)
        
#         # print(f"{residual=}")
        
#     return residual

In [8]:
# x0 = np.hstack([cal.get_pos(), cal.get_angles()])
# cc = orig_cal.int_par.cc

# sol = opt.minimize(residual, x0, args=(ref_pts, targs, control, cc), method='Nelder-Mead', tol=1e-6)

In [9]:
# print( residual(np.hstack([orig_cal.get_pos(), orig_cal.get_angles()]), ref_pts, targs, control, orig_cal.int_par.cc))

In [10]:
import copy
def added_par_residual(added_par_array, ref_pts, targs, control, cal):
    # print(calibration_array)
    # print(ref_pts)
    # print(targs)
    # print(control)
    # print(calibration_array)
    c = copy.deepcopy(cal)
    c.added_par = added_par_array
    
    # print(f"{c.get_pos()=}")
                
    residual = 0
    for i in range(len(targs)):
        xc, yc = pixel_to_metric(targs[i].x, targs[i].y, control)
        # print(f"{xc=}, {yc=} mm")
        
        xp, yp = img_coord(ref_pts[i], c, control.mm)
        # print(f"{xp=}, {yp=} mm")
        residual += ((xc - xp)**2 + (yc - yp)**2)
        
        # print(f"{residual=}")
        
    return residual

In [11]:
x0 = np.array(cal.added_par.tolist())

sol = opt.minimize(added_par_residual, x0, args=(ref_pts, targs, control, cal), method='Nelder-Mead', tol=1e-6)
print(f"{sol.x=}")
# print(sol.x - np.hstack([orig_cal.get_pos(), orig_cal.get_angles()]))



sol.x=array([-2.22546580e-03,  1.70567795e-06, -3.75579841e-10, -1.43985798e-03,
       -1.07047604e-03,  1.07511509e+00,  8.82840749e-04])


In [12]:
# print(sol.x)
print(cal.added_par)
cal.set_added_par(sol.x)
print(cal.added_par)
full_calibration(cal, ref_pts, targets, control, orient_par)
print(cal.added_par)

[0. 0. 0. 0. 0. 1. 0.]
[-2.22546580e-03  1.70567795e-06 -3.75579841e-10 -1.43985798e-03
 -1.07047604e-03  1.07511509e+00  8.82840749e-04]
[-2.22546580e-03  1.70567795e-06 -3.75579841e-10 -1.43985798e-03
 -1.07047604e-03  1.07511509e+00  8.82840749e-04]


In [13]:
print(cal.get_pos())
print(cal.get_angles())
print(cal.get_primary_point())
print(cal.get_decentering())
print(cal.added_par)

[17.52102953 16.18864526 99.01488232]
[-0.14194853  0.14975585 -0.0373757 ]
[ -2.42252617   3.22403363 100.00053252]
[-0.00143986 -0.00107048]
[-2.22546580e-03  1.70567795e-06 -3.75579841e-10 -1.43985798e-03
 -1.07047604e-03  1.07511509e+00  8.82840749e-04]
